In [1]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from math import exp
import io
from sklearn.cross_validation import train_test_split


In [2]:
filename = "iris.data.txt"
r = io.open(filename, encoding='utf8').readlines()
X = []
Y = []
for i in r[0:150]:
    x = i.split(',')
    X.append(map(float,x[0:len(x)-1]))
    if x[-1] == "Iris-setosa\n":
        Y.append(0)
    elif x[-1] == 'Iris-versicolor\n':
        Y.append(1)
    elif x[-1] == 'Iris-virginica\n':
        Y.append(2)
X = np.array(X)
Y = np.array(Y)

In [3]:
def softmax_fn(x, theta):
    y = np.unique(Y)
    softmax = []
    de = 0
    for j in range(len(y)):
        de = de + np.exp(np.dot(theta[j].transpose(),x))
    for k in range(len(y)):
        nu = np.exp(np.dot(theta[k].transpose(),x))
        softmax.append(nu/de)
    return softmax

In [4]:
s = softmax_fn(X[1],np.random.rand(3,4)/10000)
print s

[0.33333707821384012, 0.33329809828368967, 0.33336482350247015]


In [5]:
def iid(x):
    if x:
        return 1
    else:
        return 0

In [6]:
def decent(y,x,maxiterations = 800):
    eta = 0.001
    y1 = np.unique(Y)
    k = len(y1)
    v = 0
    iterations = 0
    m,n = np.shape(x)
    Theta =  np.random.rand(k,n)/10000
    while(iterations<maxiterations):
        sumv = [[0]]*len(y1)
        for j in range(len(y1)):
            for i in range(x.shape[0]):
                ii = iid(y[i]==j)
                hypo = softmax_fn(x[i],Theta)
                for l in range(len(hypo)):
                    sumv[l] += ((hypo[l] - ii)*x[i])
        for r in range(len(Theta)):
            Theta[r] = Theta[r] - (eta * sumv[r])
        iterations+=1
    return Theta

In [7]:
softie = decent(Y,X)

In [8]:
print softie

[[  4.93205376e-04   7.20545554e-04  -5.88284026e-04  -3.37878296e-04]
 [  8.81776465e-01   4.92397238e-01   4.90950770e-01   1.45731607e-01]
 [ -8.82092558e-01  -4.92973936e-01  -4.90173270e-01  -1.45335675e-01]]


In [9]:
def classify(theta,x):
    soft_list =[]
    #a = np.unique(Y)
    #for i in range(len(a)):
    ind = np.argmax(np.array(softmax_fn(x,theta)))
    return ind
        

In [10]:
cc = classify(np.random.rand(3,4)/10000,X[0])
print cc

1


In [11]:
def classifyAll(Y,X,X1):
    classifier_predict =[]
    ntheta = decent(Y,X)
    for i in range(0,X1.shape[0]):
        ind = classify(ntheta,X1[i])
        classifier_predict.append(ind)
    return classifier_predict

In [12]:
def accuracy(cm):
    return np.trace(cm)*1.0/np.sum(cm)

In [13]:
def precision(cm):
    prec = []
    for i in range(len(cm)):
        for j in range(len(cm)):
            if i == j:
                prec.append(cm[i][j]*1.0/np.sum(cm[i][j],axis=0))
    return prec

In [14]:
def recall(cm):
    rec = []
    for i in range(len(cm)):
        for j in range(len(cm)):
            if i == j:
                rec.append(cm[i][j]*1.0/np.sum(cm[i][j]))
    return rec

In [15]:
def FM(prec,rec):
    f1 = []
    for i,j in zip(prec,rec):
        f1.append(2.0*(i*j)/(i+j))
    return f1

In [16]:
def confusionmatrix(y,classifier_pred):
    clabels = [0,1,2]
    cm_fold = []
    for i in clabels:
        tmp =[0]*len(clabels)
        for j in range(len(y)):
            #print j
            if y[j] == i and y[j] == classifier_pred[j]:
                tmp[clabels.index(i)] += 1
            elif y[j] == i and y[j] != classifier_pred[j]:
                tmp[clabels.index(classifier_pred[j])] += 1
        cm_fold.append(tmp)
    return cm_fold

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

labels = classifyAll(y_train,X_train,X_test)

#classes = np.unique(Y)
cm = confusionmatrix(y_test,labels)

acc = accuracy(cm)

print cm

print acc

[[19, 0, 0], [15, 0, 0], [16, 0, 0]]
0.38


In [20]:
kf = KFold(len(Y), n_folds=10, shuffle = False)
for train_index, test_index in kf:
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    labels = classifyAll(Y_train,X_train,X_test)

    cm = confusionmatrix(Y_test,labels)


    acc = accuracy(cm)
    
    print cm
    print acc

[[0, 15, 0], [0, 0, 0], [0, 0, 0]]
0.0
[[0, 15, 0], [0, 0, 0], [0, 0, 0]]
0.0
[[15, 0, 0], [0, 0, 0], [0, 0, 0]]
1.0
[[0, 5, 0], [0, 10, 0], [0, 0, 0]]
0.666666666667
[[0, 0, 0], [0, 0, 15], [0, 0, 0]]
0.0
[[0, 0, 0], [0, 0, 15], [0, 0, 0]]
0.0
[[0, 0, 0], [0, 10, 0], [0, 5, 0]]
0.666666666667
[[0, 0, 0], [0, 0, 0], [0, 0, 15]]
1.0
[[0, 0, 0], [0, 0, 0], [0, 15, 0]]
0.0
[[0, 0, 0], [0, 0, 0], [0, 0, 15]]
1.0
